In [1]:
GPU_ID = [0, 1]
# tensorboard --logdir=/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytoh/cdiscount/logs_resnet --port=6008
# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
get_ipython().magic('env CUDA_VISIBLE_DEVICES = ')
#%env CUDA_VISIBLE_DEVICES = 0
import numpy as np
import pandas as pd
import shutil
import uuid
import math

import matplotlib.pyplot as plt
from scipy.misc import imread, imsave, imshow, imresize
import tensorflow as tf
from tensorflow.contrib.layers.python.layers import layers as layers_lib
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.01)
#, gpu_options=gpu_options
tf_sess = tf.Session(config=tf.ConfigProto(device_count={'CPU' : 1, 'GPU' : 0}, allow_soft_placement=True))
# now tensorflow will assume there not exist gpu
# use this tf_session following, and close in the end
#tf_sess.close()

import os
import os.path

import time
import io
from datetime import datetime
import gc # garbage collector
import logging
get_ipython().magic('env CUDA_VISIBLE_DEVICES = {}'.format(', '.join(map(str, GPU_ID))))
from tensorboard_logger import configure, log_value
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torch.autograd import Variable
import torch
import torch_resnet

from scipy.sparse import *
import tables as tb
import random

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
get_ipython().magic('matplotlib inline')
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
get_ipython().magic('load_ext autoreload')
get_ipython().magic('autoreload 2')

env: CUDA_VISIBLE_DEVICES=
env: CUDA_VISIBLE_DEVICES=0, 1


In [2]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/'
#PRETRAINED_MODEL_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/models/resnet101-5d3b4d8f.pth'
#PRETRAINED_MODEL_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet/Resnet_pytorch_state_49485_2017-12-04_11_10_54.pth'
PRETRAINED_MODEL_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_focal_loss/Resnet_pytorch_state_438453_2017-12-14_06_43_50.pth'


LOG_DIR = DATASET_PATH + 'pytorch/cdiscount/logs_resnet_test'
TEST_PATH = DATASET_PATH + 'Test/'
OUTPUT_PATH = DATASET_PATH + 'pytorch/cdiscount/logs_resnet_test/output_{}.csv'
CATEGORY_NAME_PATH = DATASET_PATH + 'category_names.csv'
CATEGORY_WEIGHT_PATH = DATASET_PATH + 'catogory_with_weight.csv'
PROB_SAVE_FILE = DATASET_PATH + 'pytorch/cdiscount/logs_resnet_test/probs_{}.h5'
ID_SAVE_FILE = DATASET_PATH + 'pytorch/cdiscount/logs_resnet_test/ids.csv'

NUM_OF_TOPK = 20
NUM_TO_AUG = 1
#BATCH_SIZE = 5//NUM_TO_AUG
BATCH_SIZE = 2048//NUM_TO_AUG
IMAGE_WIDTH = 180
IMAGE_HEIGHT = 180
NUM_CLASS = 5270
LEVEL1_CLASS = 49
LEVEL2_CLASS = 483
TOTAL_EXAMPLES = 3095080
INPUT_THREADS = 1
NUM_STEPS = int(TOTAL_EXAMPLES / BATCH_SIZE) + 1

log_step_interval = 30

In [3]:
def set_logging(logger_name, logger_file_name):
    log = logging.getLogger(logger_name)
    log.setLevel(logging.DEBUG)

    # create formatter and add it to the handlers
    print_formatter = logging.Formatter('%(message)s')
    file_formatter = logging.Formatter('%(asctime)s - %(name)s_%(levelname)s: %(message)s')

    # create file handler which logs even debug messages
    fh = logging.FileHandler(logger_file_name, mode='w')
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(file_formatter)
    log.addHandler(fh)
    # both output to console and file
    consoleHandler = logging.StreamHandler()
    consoleHandler.setFormatter(print_formatter)
    log.addHandler(consoleHandler)
    
    return log

In [4]:
log = set_logging('CDiscount', DATASET_PATH + 'pytorch/cdiscount/resnet_pytorch_test.log')
log.info('here is an info message.')

here is an info message.


In [5]:
LIB_NAME = 'segment_median'

def load_op_module(lib_name):
    """
    Load TensorFlow operator library.
    """
    # use absolute path so that ops.py can be called from other directory
    lib_path =  '../lib{0}.so'.format(lib_name)
    # duplicate library with a random new name so that
    # a running program will not be interrupted when the original library is updated
    lib_copy_path = '/tmp/lib{0}_{1}.so'.format(str(uuid.uuid4())[:8], LIB_NAME)
    shutil.copyfile(lib_path, lib_copy_path)
    oplib = tf.load_op_library(lib_copy_path)
    return oplib

segment_median = load_op_module(LIB_NAME).segment_median

In [6]:
class TimeRecorder(object):
    def __init__(self):
        super(TimeRecorder, self).__init__()
        self._tick_map = dict()
        self._recorder = dict()
        self._use_time = dict()
        self._last_interval = dict()
        self.register_event('ticks', 1, True)
    # register at the very begining
    # when not use time, call reset_event with your initial value at each start
    def register_event(self, name, tick, use_time=True):
        self._tick_map[name] = tick
        self._last_interval[name] = 0
        self._recorder[name] = 0
        if use_time:
            self._recorder[name] = time.time() 
        self._use_time[name] = use_time
    def reset_event(self, name, criterion=None):
        if self._use_time[name]: self._recorder[name] = time.time()
        elif criterion is not None: self._recorder[name] = criterion
        self._last_interval[name] = 0
    def cancel_event(self, name):
        self._tick_map.pop(name, None)
        self._last_interval.pop(name, None)
        self._recorder.pop(name, None)
        self._use_time.pop(name, None)
    def get_ticks_passed(self):
        passed = time.time() - self._recorder['ticks']
        self._recorder['ticks'] = time.time()
        return passed/self._tick_map['ticks']
    # you can call this to get how log elapsed after you get True from check_for_me
    def how_long_before(self, name):
        return self._last_interval[name]
    def check_for_me(self, name, criterion=None):
        if self._use_time[name]:
            if time.time() - self._recorder[name] >= self._tick_map[name]:
                self._last_interval[name] = time.time() - self._recorder[name]
                self._recorder[name] = time.time()
                return True
        elif criterion is not None:
            if criterion - self._recorder[name] >= self._tick_map[name]:
                self._last_interval[name] = criterion - self._recorder[name]
                self._recorder[name] = criterion
                return True
        return False
def load_pretrain_file(net, pretrain_file, skip=[]):
    pretrain_state_dict = torch.load(pretrain_file, map_location=lambda storage, loc: storage)
    state_dict = net.state_dict()
    keys = list(state_dict.keys())
    #print(pretrain_state_dict.keys())
    for key in keys:
        if any(s in key for s in skip):
            continue
        pretrain_key = key# 'module.' + key
        #if 'layer0.0.conv.' in key: pretrain_key=key.replace('layer0.0.conv.',  'conv1.' )
        state_dict[key] = pretrain_state_dict[pretrain_key]

    net.load_state_dict(state_dict)
    return net

In [7]:
class LabelMapping(object):
    def __init__(self, catogory_file_path):
        super(LabelMapping, self).__init__()
        self._category_level_csv = catogory_file_path
        self._category_map = self.cvt_csv2tfrecord()
        self._mapping_strings = tf.constant( [ str(key) for key in self._category_map.keys() ] )
        self._index2catogory_inv_table = tf.contrib.lookup.index_to_string_table_from_tensor(self._mapping_strings, default_value="0000000000")
       
    @property
    def category_map(self):
        return self._category_map
    @property
    def index2catogory_inv_table(self):
        return self._index2catogory_inv_table
   
    def cvt_csv2tfrecord(self):
        count = 0
        category_map = dict()
        csv = pd.read_csv(self._category_level_csv).values
        for row in csv:  
            category_id = row[0]
            category_map[category_id] = count
            count += 1

        return category_map

In [8]:
class CdiscountDataset(Dataset):
    def __init__(self, sess, data_path, file_begin_match, num_examples, num_classes, buffer_size, batch_size, is_training):
        super(CdiscountDataset, self).__init__()
        self._data_file_list = [ os.path.join(data_path, x) for x in os.listdir(data_path) if lambda x: os.path.isfile(x) and file_begin_match in x ]
        self._num_examples = num_examples
        self._tf_sess = sess
        self._num_classes = num_classes
        self._batch_size = batch_size
        self._buffer_size = buffer_size
        self._is_training = is_training
    def __len__(self):
        return self._num_examples

    @staticmethod
    def image_normalized(image):
        mean = [0.485, 0.456, 0.406 ]
        std  = [0.229, 0.224, 0.225 ]

        image = image.transpose((2,0,1))
        image = image.astype(float)/255.
        
        image[0] = (image[0] - mean[0]) / std[0]
        image[1] = (image[1] - mean[1]) / std[1]
        image[2] = (image[2] - mean[2]) / std[2]

        return image.astype(np.float32)

    @staticmethod
    def fix_center_crop(image, size=(160,160)):

        height, width = image.shape[0:2]
        w,h = size

        x0 = (width  -w)//2
        y0 = (height -h)//2
        x1 = x0 + w
        y1 = y0 + h
        image = image[y0:y1, x0:x1]

        return image
    @staticmethod
    def random_horizontal_flip(image, u=0.5):
        if random.random() < u:
            image = np.flip(image,1)  #np.fliplr(img) ##left-right
        return image
    @staticmethod
    def random_crop(image, size=(160,160), u=0.5):

        height,width=image.shape[0:2]
        w,h = size

        if random.random() < u:
            x0 = np.random.choice(width - w)
            y0 = np.random.choice(height - h)
        else:
            x0 = (width  -w)//2
            y0 = (height -h)//2

        x1 = x0 + w
        y1 = y0 + h
        image = image[y0:y1, x0:x1]

        return image
    @staticmethod
    def random_resize(image, scale_x_limits=[0.9, 1.1], scale_y_limits=[0.9, 1.1], u=0.5):
        if random.random() < u:
            height,width=image.shape[0:2]

            scale_x  = random.uniform(scale_x_limits[0],scale_x_limits[1])
            if scale_y_limits is not None:
                scale_y  = random.uniform(scale_y_limits[0],scale_y_limits[1])
            else:
                scale_y = scale_x

            w = int(scale_x*width )
            h = int(scale_y*height)

            image = imresize(image,(h,w))
        return image

    @staticmethod
    def _preprocess_by_pytorch_train(image):
        image = CdiscountDataset.random_resize(image, scale_x_limits=[0.9,1.1], scale_y_limits=[0.9,1.1], u=0.5)
        # flip  random ---------
        image = CdiscountDataset.random_crop(image, size=(160,160), u=0.5) 
        image = CdiscountDataset.random_horizontal_flip(image, u=0.5)
        return image.astype(np.float32)
    @staticmethod
    def _preprocess_by_pytorch_test_with_aug(image):
        num_batch_size = image.shape[0]
        #print(num_batch_size)
        list_of_images = np.split(image, num_batch_size)
        return [CdiscountDataset._preprocess_by_pytorch_train(_image) for _image in list_of_images]
    @staticmethod
    def _preprocess_by_pytorch_test(image):
        image  = CdiscountDataset.fix_center_crop(image, size=(160,160))  
        return image.astype(np.float32)
    @staticmethod
    def _preprocess_normalize(image):
        return CdiscountDataset.image_normalized(image)
    @staticmethod
    def _array_to_image_transform(image):
        mean = [0.485, 0.456, 0.406 ]
        std  = [0.229, 0.224, 0.225 ]
        if random.random() < 0.00001:
            image_to_save = image
            image_to_save[0] = image_to_save[0]*std[0] + mean[0]
            image_to_save[1] = image_to_save[1]*std[1] + mean[1]
            image_to_save[2] = image_to_save[2]*std[2] + mean[2]

            image_to_save = image_to_save*255
            image_to_save = np.transpose(image_to_save, (1, 2, 0))
            image_to_save = image_to_save.astype(np.uint8)
            imsave('/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/Debug/{}.jpg'.format(int(random.random()*100000)), image_to_save)
        return image
    @staticmethod
    def extern_array_to_image_transform(image, filepath):
        mean = [0.485, 0.456, 0.406 ]
        std  = [0.229, 0.224, 0.225 ]
        image_to_save = np.array(image)
        image_to_save[0] = image_to_save[0]*std[0] + mean[0]
        image_to_save[1] = image_to_save[1]*std[1] + mean[1]
        image_to_save[2] = image_to_save[2]*std[2] + mean[2]

        image_to_save = image_to_save*255
        image_to_save = np.transpose(image_to_save, (1, 2, 0))
        image_to_save = image_to_save.astype(np.uint8)
        imsave(filepath, image_to_save)

    def _parse_function(self, example_proto):
        features = {'img_raw': tf.FixedLenFeature([], tf.string, default_value=''),
                    'product_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))}
                
        parsed_features = tf.parse_single_example(example_proto, features)
        image = tf.image.decode_image(parsed_features["img_raw"])
        product_label = parsed_features["product_id"]
        
        image_to_aug = tf.tile(tf.expand_dims(image, 0), tf.constant([NUM_TO_AUG, 1, 1, 1]))
        product_id_to_aug = tf.tile(tf.expand_dims(product_label, 0), tf.constant([NUM_TO_AUG]))

        image_list = [tf.squeeze(image, 0) for image in tf.split(image_to_aug, tf.ones_like(product_id_to_aug), 0)]
        #image_list = tf.unstack(image_to_aug)

        if self._is_training:
            image_preprocessed_list = [tf.py_func(CdiscountDataset._preprocess_by_pytorch_train, [image], tf.float32, stateful=True) for image in image_list]
        else:
            image_preprocessed_list = [tf.py_func(CdiscountDataset._preprocess_by_pytorch_test, [image], tf.float32, stateful=False) for image in image_list]
        image_preprocessed_list = [tf.py_func(CdiscountDataset._preprocess_normalize, [image], tf.float32, stateful=False) for image in image_preprocessed_list]
        #image_preprocessed_list = [tf.py_func(CdiscountDataset._array_to_image_transform, [image], tf.float32, stateful=False) for image in image_preprocessed_list]
        
        return tf.stack(image_preprocessed_list), product_id_to_aug
#     def _parse_function(self, example_proto):
#         features = {'img_raw': tf.FixedLenFeature([], tf.string, default_value=''),
#                     'product_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))}
                
#         parsed_features = tf.parse_single_example(example_proto, features)
#         image = tf.image.decode_image(parsed_features["img_raw"])
#         product_label = parsed_features["product_id"]
        
#         image_to_aug = tf.tile(tf.expand_dims(image, 0), tf.constant([NUM_TO_AUG, 1, 1, 1]))
#         product_id_to_aug = tf.tile(tf.expand_dims(product_label, 0), tf.constant([NUM_TO_AUG]))

#         image_preprocessed_list = tf.py_func(CdiscountDataset._preprocess_by_pytorch_test_with_aug, [image_to_aug], [tf.float32]*NUM_TO_AUG, stateful=True)
        
#         image_preprocessed_list = [tf.py_func(CdiscountDataset._preprocess_normalize, [image], tf.float32, stateful=False) for image in image_preprocessed_list]
#         #image_preprocessed_list = [tf.py_func(CdiscountDataset._array_to_image_transform, [image], tf.float32, stateful=False) for image in image_preprocessed_list]

#         return tf.stack(image_preprocessed_list), product_label
    
    
    def get_next(self):
        batch_images_aug, batch_id_aug = self._next_iter
        batch_images = tf.reshape(batch_images_aug, [-1, 3, 160, 160])
        batch_id = tf.reshape(batch_id_aug, [-1])
        return batch_images, batch_id
    def create_dataset(self):
        self._dataset = tf.data.TFRecordDataset(self._data_file_list, compression_type='ZLIB', buffer_size = 409600)
        parse_func = lambda example : self._parse_function(example)
        self._dataset = self._dataset.map(parse_func, num_parallel_calls=INPUT_THREADS)
        self._dataset = self._dataset.prefetch(self._batch_size * 3)
        self._dataset = self._dataset.batch(self._batch_size)
        self._iterator = self._dataset.make_initializable_iterator()
        self._next_iter = self._iterator.get_next()

        return self._iterator.initializer


In [9]:
def slices_to_dims(slice_indices):
    """
    Args:
    slice_indices: An [N, k] Tensor mapping to column indices.
    Returns:
    An index Tensor with shape [N * k, 2], corresponding to indices suitable for
    passing to SparseTensor.
    """
    slice_indices = tf.cast(slice_indices, tf.int64)
    num_rows = tf.shape(slice_indices, out_type=tf.int64)[0]
    row_range = tf.range(num_rows)
    # row_range expanded from [num_rows] into [num_rows, 1]
    # every item in k_th row of slice_indices are multiplied by num_rows, then added by k with broadcast
    item_numbers = slice_indices * num_rows + tf.expand_dims(row_range, axis=1)
    # flaten so that each row represent each element
    item_numbers_flat = tf.reshape(item_numbers, [-1])
    # convert back by zip op
    return item_numbers_flat % num_rows, item_numbers_flat // num_rows

timer_holder = TimeRecorder()
timer_holder.register_event('eval_log_time', log_step_interval, False)

cdiscount_net = torch_resnet.resnet152(pretrained=False, num_classes=NUM_CLASS)

#print(cdiscount_net)
if torch.cuda.is_available():
    cdiscount_net = cdiscount_net.cuda()
for p in cdiscount_net.parameters():
    p.requires_grad = False
if len(GPU_ID) > 1:
    cdiscount_net = torch.nn.DataParallel(cdiscount_net) #use default
cdiscount_net = load_pretrain_file(cdiscount_net, PRETRAINED_MODEL_PATH, [])

# we use train augumentation to diversify the input images
test_set = CdiscountDataset(tf_sess, TEST_PATH, 'output_file', TOTAL_EXAMPLES, NUM_CLASS, 12000, BATCH_SIZE, False)

test_initializer = test_set.create_dataset()

# when you first create a session, there will be one default graph with it unless you pass one another
tf_sess.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(), tf.tables_initializer()))
default_graph = tf.get_default_graph()
additional_graph = tf.Graph()
tf_additional_sess = tf.Session(graph = additional_graph, config=tf.ConfigProto(device_count={'CPU' : 1, 'GPU' : 0}, allow_soft_placement=True))

with tf.device('/cpu:0'):
    with additional_graph.as_default():
        label_mapping = LabelMapping(CATEGORY_NAME_PATH)
        last_prob = tf.placeholder(tf.float32)
        last_id = tf.placeholder(tf.int64)
        batch_id = tf.placeholder(tf.int64, shape=(None))
        logits_pred = tf.placeholder(tf.float32, shape=(None, 5270))

        test_probabilities = layers_lib.softmax(logits_pred, scope='torch_logits2prob')
        predictions = tf.argmax(test_probabilities, 1)

        test_predictions_values = tf.string_to_number(label_mapping.index2catogory_inv_table.lookup(predictions), out_type=tf.int64) 

        top_values, top_indices = tf.nn.top_k(test_probabilities, k = NUM_OF_TOPK, sorted=True)
        (row_indice, col_indice), value_array = slices_to_dims(top_indices), tf.reshape(top_values, [-1])
        cur_prob_shape = tf.shape(test_probabilities)
        # concat betweent batches
        _, idx, count = tf.unique_with_counts(batch_id)
        #print(tf.dynamic_partition(batch_id, tf.not_equal(idx, tf.shape(count)[0] - 1).eval(), 2)[1].eval())  
        cur_id_tail, _cur_id_head = tf.dynamic_partition(batch_id, tf.cast(tf.not_equal(idx, tf.shape(count)[0] - 1), tf.int32), 2)
        with tf.control_dependencies([cur_id_tail, _cur_id_head]):
            cur_id_head = tf.concat([last_id, _cur_id_head], axis = 0)
        #cur_id_head = tf.concat([last_id, tf.concat(tf.split(batch_id, count)[0:-1], axis = 0)], axis = 0)
        #cur_id_tail = tf.split(batch_id, count)[-1]

        cur_prob_tail, _cur_prob_head = tf.dynamic_partition(test_probabilities, tf.cast(tf.not_equal(idx, tf.shape(count)[0] - 1), tf.int32), 2)
        with tf.control_dependencies([last_prob, _cur_prob_head]):
            cur_prob_head = tf.concat([last_prob, _cur_prob_head], axis = 0)
        #cur_prob_head = tf.concat([last_prob, tf.concat(tf.split(test_probabilities, count[0:-1]), axis = 0)], axis = 0)
        #cur_prob_tail = tf.split(test_probabilities, count)[-1]
        with tf.control_dependencies([cur_id_head, cur_prob_head]):
            raw_id, idx, _ = tf.unique_with_counts(cur_id_head)
            mean_prob = tf.segment_mean(cur_prob_head, idx)
            #mean_prob = segment_median(cur_prob_head, tf.cast(idx, tf.float32))
            mean_label = tf.string_to_number(label_mapping.index2catogory_inv_table.lookup(tf.argmax(mean_prob, 1)), out_type=tf.int64) 
            #mean_label = tf.argmax(mean_prob, 1)
        with tf.control_dependencies([mean_prob, mean_label]):
            #last_id = cur_id_tail
            #last_prob = cur_prob_tail
            # last partition may have nothing to concat
            raw_id_tail, idx_tail, _ = tf.unique_with_counts(cur_id_tail)
            mean_prob_tail = tf.segment_mean(cur_prob_tail, idx_tail)
            #mean_prob_tail = segment_median(cur_prob_tail, tf.cast(idx_tail, tf.float32))
            tail_label = tf.string_to_number(label_mapping.index2catogory_inv_table.lookup(tf.argmax(mean_prob_tail, 1)), out_type=tf.int64) 
        #total_pred = tf.string_to_number(label_mapping.index2catogory_inv_table.lookup(tf.argmax(mean_prob_tail, 1)), out_type=tf.int64) 
        tf_additional_sess.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(), tf.tables_initializer()))

#tf_sess.run(train_initializer)
#print(tf_sess.run(train_set.get_next()))
#tf_sess.run(train_set[0])
        
lats_pred = []
last_batch_id = []
last_feed_id = np.empty([0])
last_feed_prob = np.empty([0, NUM_CLASS])
save_file_name = OUTPUT_PATH.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

save_logits_file_name = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/{}.csv'.format(datetime.now().strftime('logits_%Y-%m-%d %H:%M:%S'))

prob_save_file_name = PROB_SAVE_FILE.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

hdf5_file = tb.open_file(prob_save_file_name, 'w')
total_prob_store = hdf5_file.create_earray(hdf5_file.root, 'prob', tb.Float32Atom(), shape=(0,), filters=tb.Filters(complevel=5, complib='zlib'))
total_row_indice = hdf5_file.create_earray(hdf5_file.root, 'row', tb.Int64Atom(), shape=(0,), filters=tb.Filters(complevel=5, complib='zlib'))
total_col_indice = hdf5_file.create_earray(hdf5_file.root, 'col', tb.Int64Atom(), shape=(0,), filters=tb.Filters(complevel=5, complib='zlib'))
log.info(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    
#target_onehot = torch.FloatTensor(batch_size, 4)
timer_holder.reset_event('eval_log_time', 0)
MDEBUG = False

try:
#if True:
    # check on validation every epoches            
    cdiscount_net.eval()
    tf_sess.run(test_initializer)
    timer_holder.reset_event('eval_log_time', 1)
    #if False:
    for index in range(len(test_set)):
        try:
            data_value = tf_sess.run(test_set.get_next())
            #next_example = [torch.from_numpy(array) for array in data_value]
            next_example, next_product_id = torch.FloatTensor(data_value[0]), torch.LongTensor(data_value[1])
        except tf.errors.OutOfRangeError:
            log.info('dataset run out.')
            break

        if torch.cuda.is_available():
            example_in = Variable(next_example).cuda()
        else:
            example_in = Variable(next_example)
        #logits_from_pytorch = Variable(torch.rand(next_product_id.size()[0], 5270))
        logits_from_pytorch = cdiscount_net(example_in)
        _, batch_top1 = torch.topk(nn.Softmax()(logits_from_pytorch), 1)
        batch_top1 = batch_top1.type(torch.LongTensor)
        if torch.cuda.is_available():
            batch_top1 = batch_top1.cuda()
        #print(batch_top1.data.cpu().numpy())
        
        total_pred, last_feed_id, last_feed_prob, test_pred, test_prob, test_batch_id, lats_pred, last_batch_id, = \
                            tf_additional_sess.run([test_predictions_values, cur_id_tail, cur_prob_tail, mean_label, mean_prob, \
                                      raw_id, tail_label, raw_id_tail], feed_dict = {last_prob: last_feed_prob, last_id: last_feed_id, batch_id: data_value[1], logits_pred: logits_from_pytorch.data.cpu().numpy()})
        #print(csr_matrix((sparse_value, (sparse_row, sparse_col)), shape=sparse_shape).toarray())
        #total_prob_store[last_row_to_save:last_row_to_save + sparse_shape[0],:] = csr_matrix((sparse_value, (sparse_row, sparse_col)), shape=sparse_shape).toarray()
        
#         print(total_pred)
#         print(len(total_pred))
        raw_pred = pd.DataFrame(data=total_pred, index=data_value[1])
        if not os.path.isfile(save_logits_file_name):
            raw_pred.to_csv(save_logits_file_name, mode='a', index=True, sep=',')
        else:
            raw_pred.to_csv(save_logits_file_name, mode='a', index=True, sep=',', header=False)

        #logits_df = pd.DataFrame(data=logits_from_pytorch.data.cpu().numpy(), index=data_value[1])

#         if not os.path.isfile(save_file_name):
#             logits_df.to_csv(save_logits_file_name, mode='a', index=True, sep=',', compression='bz2')
#         else:
#             logits_df.to_csv(save_logits_file_name, mode='a', index=True, sep=',', header=False, compression='bz2')


#         total_prob_store.append(sparse_value)
#         total_row_indice.append(sparse_row)
#         total_col_indice.append(sparse_col)
        
#         print(len(test_batch_id), len(test_pred))
#         print(test_batch_id)
        #print(test_pred)
        #last_row_to_save += sparse_shape[0]
        #print(len(test_prob[0]))
        df = pd.DataFrame({'_id' : test_batch_id, 'category_id' : test_pred})

        if not os.path.isfile(save_file_name):
            df.to_csv(save_file_name, mode='a', index=False, sep=',')
        else:
            df.to_csv(save_file_name, mode='a', index=False, sep=',', header=False)
#         log.info('BB ID: {}'.format(data_value[1]))
#         log.info('Test Label: {}'.format(test_pred))
#         log.info('Test Prob: {}'.format(test_prob))
#         log.info('Test Ids: {}'.format(test_batch_id))
#         log.info('Last Label: {}'.format(lats_pred))
#         #log.info('Test Prob: {}'.format(test_prob))
#         log.info('Last Ids: {}'.format(last_batch_id))
#         log.info('last_feed_id: {}'.format(last_feed_id))
#         log.info('last_feed_prob: {}'.format(last_feed_prob))
#         if index > 3:
#             break

        if timer_holder.check_for_me('eval_log_time', index):
            time_passed = timer_holder.get_ticks_passed()
            log.info('####### testting logging #######')
            log.info('\ttest/current_step: {}/{}'.format(index, NUM_STEPS))
            log.info('\ttest/sec_per_step: {:.3f}'.format(time_passed/log_step_interval))
            log.info('\ttest/roughly {:6.3f} hours to go.'.format(  time_passed/log_step_interval*( (NUM_STEPS-index) > 0 and (NUM_STEPS-index)/3600. or 0.001 )  ))
            #log.info('Test Label: {}'.format(test_pred))
            #log.info('Test Prob: {}'.format(test_prob))
            #log.info('Test Ids: {}'.format(test_batch_id))
            if MDEBUG: break
except BaseException as error:
#if False:
    # use tensorflow to fill the GPU no matter what reason caused
    log.info("Error raised: {}.\r\nTestting did't finished.".format(error))
    tf_sess.close()
    get_ipython().magic('env CUDA_VISIBLE_DEVICES = {}'.format(', '.join(map(str, GPU_ID))))
    import tensorflow as tf
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.)
    tf_sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options))
tf_sess.close()
df = pd.DataFrame({'_id' : last_batch_id, 'category_id' : lats_pred})
df.to_csv(save_file_name, mode='a', index=False, sep=',', header=False)
hdf5_file.close()
log.info('Finished Testting: {}'.format(datetime.now().strftime('%Y-%m-%d_%H_%M_%S')))

2017-12-14 20:17:14
####### testting logging #######
	test/current_step: 31/1512
	test/sec_per_step: 4.240
	test/roughly  1.744 hours to go.
####### testting logging #######
	test/current_step: 61/1512
	test/sec_per_step: 3.632
	test/roughly  1.464 hours to go.
####### testting logging #######
	test/current_step: 91/1512
	test/sec_per_step: 3.675
	test/roughly  1.451 hours to go.
####### testting logging #######
	test/current_step: 121/1512
	test/sec_per_step: 3.766
	test/roughly  1.455 hours to go.
####### testting logging #######
	test/current_step: 151/1512
	test/sec_per_step: 3.714
	test/roughly  1.404 hours to go.
####### testting logging #######
	test/current_step: 181/1512
	test/sec_per_step: 3.954
	test/roughly  1.462 hours to go.
####### testting logging #######
	test/current_step: 211/1512
	test/sec_per_step: 3.903
	test/roughly  1.411 hours to go.
####### testting logging #######
	test/current_step: 241/1512
	test/sec_per_step: 3.848
	test/roughly  1.359 hours to go.
#######

Finished Testting: 2017-12-14_21_52_35


env: CUDA_VISIBLE_DEVICES=0, 1


In [10]:
test_data = pd.read_csv(save_file_name)
log.info(test_data.tail())

              _id  category_id
1767848  23597507   1000005732
1767849  23597584   1000000504
1767850  23597618   1000010151
1767851  23597686   1000011427
1767852  23597750   1000006458


In [ ]:
print(df)
print(pd.read_csv('/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/output_2017-12-14 18:32:02.csv'))

In [ ]:
print(pd.read_csv('/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/output_2017-12-14 14:35:36.csv'))

In [13]:
print(pd.read_csv('/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/result_mean.csv'))
print(pd.read_csv('/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/result_mean.csv')['_id'][1767852])

              _id  category_id
0              10   1000005605
1              27   1000022508
2              67   1000005986
3              94   1000004079
4             115   1000002786
5             141   1000004079
6             165   1000010653
7             196   1000010653
8             210   1000014026
9             231   1000016662
10            254   1000005874
11            272   1000014245
12            285   1000012509
13            298   1000005926
14            326   1000010653
15            338   1000010653
16            353   1000006016
17            365   1000010667
18            370   1000010653
19            386   1000010667
20            402   1000002194
21            421   1000014154
22            425   1000010653
23            441   1000014311
24            456   1000010667
25            483   1000001840
26            488   1000004141
27            499   1000011046
28            512   1000005414
29            525   1000010667
...           ...          ...
1768152 

In [ ]:
print(pd.concat([pd.read_csv(save_file_name)[:1768182], pd.read_csv('/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/0.6837.csv')[1767853:]], axis=0))

In [11]:
pd.concat([pd.read_csv(save_file_name)[:1768182], pd.read_csv('/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/0.6837.csv')[1767853:]], axis=0).to_csv('/media/rs/0E06CD1706CD0127/Kapok/kaggle/pytorch/cdiscount/logs_resnet_test/result_mean.csv', index=False)

In [ ]:
print(pd.read_csv(save_file_name)[:1768182])

In [ ]:
pd.read_csv(save_file_name)[:1768182].to_csv(save_file_name+'fff', index=False, sep=',', header=True)